In [ ]:
#!pip install librosa
#!pip install mido 
import librosa
import mido 
from mido import MidiFile, MidiTrack, Message
from scipy import signal
import numpy as np
from scipy.signal import find_peaks

In [ ]:
# Load the WAV file
wav_file = '../audio/alto.wav'
audio, sample_rate = librosa.load(wav_file)

# Extract Pitch and Timing Information
def extract_pitch_and_timing(audio, sample_rate):
    pitches, magnitudes = librosa.piptrack(y=audio, sr=sample_rate)
    pitch_peaks, _ = find_peaks(pitches[0])
    timing = pitch_peaks / sample_rate

    return pitches, timing

pitches, timing = extract_pitch_and_timing(audio, sample_rate)

# Create MIDI Notes
def create_midi_notes(pitches, timing):
    midi_notes = []

    if timing.size == 0:  # Check if the 'timing' array is empty
        return midi_notes  # Return an empty list in this case

    for i in range(len(pitches)):
        note = int(pitches[0][i])
        start_time = timing[i]
        end_time = timing[i + 1] if i + 1 < len(timing) else timing[-1]
        duration = end_time - start_time

        midi_notes.append({
            'note': note,
            'start_time': start_time,
            'duration': duration,
            'velocity': 64  # Adjust the velocity as needed
        })

    return midi_notes

midi_notes = create_midi_notes(pitches, timing)

# Create a MIDI File
def create_midi_file(midi_notes):
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)

    for note in midi_notes:
        note_on = Message('note_on', note=note['note'], velocity=note['velocity'])
        note_off = Message('note_off', note=note['note'], velocity=note['velocity'], time=int(note['duration'] * 480))

        track.append(note_on)
        track.append(note_off)

    return mid

midi_file = create_midi_file(midi_notes)

# Save the MIDI File
output_midi_file = 'output.mid'
midi_file.save(output_midi_file)